In [1]:
import lib.dongle_cmds as d
import time

In [2]:
with open('ISL69269-1 0x60.hex') as file:
    file_contents = file.read()

In [3]:
header_dict  = {'type': (0,2),   'bytenum': (2,4),
                'addr': (4,6),   'IDcode' : (6,8),
                'dum' : (8,12),  'IDdata' : (8,16),
                'zero': (14,16), 'CRC'    : (16,18)}

write7_parse = {'zero': (0,2),  'bytenum': (2,4),
                'addr': (4,6),  'comcode': (6,8),
                'data': (8,16), 'CRC'    : (16,18)}

write5_parse = {'zero': (0,2),  'bytenum': (2,4),
                'addr': (4,6),  'comcode': (6,8),
                'data': (8,14), 'CRC'    : (14,16)}

hex_file_line_parse = {'header'  : (0,5),
                       'commands': (5,-2),
                       'exec'    : (-2,-1)}

register = {'avail_NVM_slots': {'d_func': d.read_dma_cmd,'reg': ['0xc2', '0x00'],'data_length': 4, 'parse_idx':(0,1), 'mask': 0xFF, 'function': 'int_8bit'},
            'device_ID'      : {'d_func': d.read_smb_cmd,'reg': ['0xad'],        'data_length': 4, 'parse_idx':(2,3), 'mask': 0xFF, 'function': 'read_reg'},
            'revision'       : {'d_func': d.read_smb_cmd,'reg': ['0xae'],        'data_length': 5, 'parse_idx':(1,5), 'mask': 0xFF, 'function': 'rev_reg'},
            'prog_status'    : {'d_func': d.read_dma_cmd,'reg': ['0x07', '0x07'],'data_length': 4, 'parse_idx':(0,1), 'mask': 0xFF, 'function': 'int_8bit'},
            'bank_status'    : {'d_func': d.read_dma_cmd,'reg': ['0x09', '0x07'],'data_length': 4, 'parse_idx':(0,4), 'mask': 0xFF, 'function': 'read_reg'},
            'config_id'      : {'d_func': d.read_dma_cmd,'reg': ['0xc1', '0x00'],'data_length': 4, 'parse_idx':(0,1), 'mask': 0xFF, 'function': 'read_reg'},
            'crc'            : {'d_func': d.read_dma_cmd,'reg': ['0x3f', '0x00'],'data_length': 4, 'parse_idx':(0,4), 'mask': 0xFF, 'function': 'rev_reg'},
            
            'load_FW'               : {'d_func': d.write_smb_cmd,'reg':['0xe6'], 'data': ['0x10','0x00'], 'function': 'write_reg'},
            'recomp_OTP_bounds'     : {'d_func': d.write_smb_cmd,'reg':['0xe6'], 'data': ['0x06','0x28'], 'function': 'write_reg'},
            'wake_FW'               : {'d_func': d.write_smb_cmd,'reg':['0xe6'], 'data': ['0x03','0x00'], 'function': 'write_reg'},
            'apply_project_settings': {'d_func': d.write_smb_cmd,'reg':['0xe7'], 'data': ['0x01','0x00'], 'function': 'write_reg'},
            'restart_FW'            : {'d_func': d.write_smb_cmd,'reg':['0xe6'], 'data': ['0x07','0x00'], 'function': 'write_reg'},
            'restcfg'               : {'d_func': d.write_smb_cmd,'reg':['0xf2'], 'data': ['0x00'],        'function': 'write_reg'}}



In [4]:
lines        = file_contents.split('\n')
header       = lines[:5]
commands     = lines[5:-2]
exec_command = lines[-2]

In [6]:
SMB_Add = 0x60

In [7]:
#readsmb  - reg, data_length, dev_addr
#writesmb - reg, data_length, data, dev_addr  
#readdma  - reg,data_length, dev_addr

#writedma - reg, data_length=2, data, dev_addr  #same as writesmb?

In [24]:
class Command:
    def __init__(self,reg_key, dev_addr):
        self.dev_addr = dev_addr
        self.params = register[reg_key]
        self.dfunc       = self.params['d_func']
        self.reg         = self.params['reg']
        self.functiondict  = {'read_reg'    : self.read_reg,   'rev_reg'  : self.rev_reg, 
                              'int_8bit'    : self.int_8bit,   'bin32'    : self.bin32, 
                              'apply_mask'  : self.apply_mask, 'write_reg': self.write_reg}
        
    def read_reg(self):
        a,b = self.params['parse_idx']
        return self.dfunc(self.reg,self.params['data_length'],self.dev_addr)[a:b]
    
    def rev_reg(self):
        data = self.read_reg()
        data.reverse()
        return data
    
    def int_8bit(self):
        data = self.read_reg()
        return int(data[0],16)
    
    def decimal(self,data):
        return d.bytearray2decimal(data)
        
    def bin32(self,data):
        pass
    
    def apply_mask(self,data):
        data = self.read_reg()
        #convert to integer
        return data & self.params['mask']
    
    def formatted(self):
        return self.functiondict[self.params['function']]()
    
    def write_reg(self):
        data = self.params['data']
        data_length = len(data)
        self.dfunc(self.reg,data_length,data,self.dev_addr)
        
        
    
    

In [9]:
a = Command('device_ID',SMB_Add)

In [10]:
b=a.formatted() #rw_reg()#decimal(a.rw_reg()) #mask #apply_mask(a.int_8bit(a.rw_reg()))
print(b)

['0x55']


In [11]:
def parse_line(line,position_tuple):
    start, stop = position_tuple
    return line[start:stop]

def is_header(line):
    return parse_line(line,header_dict['type']) == '49'

def data_2_list(data):
    datalist = [data[i:i+2] for i in range(0, len(data), 2)]
    datalist.reverse()
    return datalist

def check_ID():
    file_id = parse_line(header[0],header_dict['IDdata'])[4:6]
    ic_id = Command('device_ID',SMB_Add).formatted()[0][2:4]
    return file_id==ic_id

def check_REV(config_or_firmware = 'config'):
    def element_cmp(a,b):
        a,b = int(a,16), int(b,16)
        return a == b 
    def cmp(a,b):
        result = [element_cmp(i,j) for i,j in zip(a,b)]
        if config_or_firmware == 'config':
            result[3] = a > b
        else:
            result[3] = a == b
        return all(result)
    file_rev = parse_line(header[1],header_dict['IDdata'])
    file_rev = [file_rev[i:i+2] for i in range(0, len(file_rev), 2)]
    ic_rev = Command('revision',SMB_Add).formatted()
    return ic_rev #cmp(ic_rev,file_rev)
  


class Cmdline:
    def __init__(self,line):
        #self.numbytes = int(line[2:4],16)
        self.pmbaddr  = hex(int(line[4:6],16)>>1)
        self.cmd      = line[6:8]
        self.data = data_2_list(line[8:-2]) 
        self.data.reverse()
        
    def write(self):
        d.write_smb_cmd(int(self.cmd,16),len(self.data),self.data,int(self.pmbaddr,16)) 


    
def load_commands(commandlist):
    for line in commandlist:
        command=Cmdline(line)
        command.write()
        time.sleep(.02)
        
def write_to_OTP(lastline):
    command=Cmdline(lastline)
    command.write()
    

    

    
class Enable:
    def __init__(self):
        pass #self.on_state = d.read_smb_cmd(0x01,1,SMB_Add)
        
    def on(self):
        d.write_smb_cmd(0x01,1,['0x80'],SMB_Add)
        time.sleep(0.25)
        state = d.read_smb_cmd(0x01,1,SMB_Add)
        print(state)
    
    def off(self):
        d.write_smb_cmd(0x01,1,['0x00'],SMB_Add)
        time.sleep(0.25)
        state = d.read_smb_cmd(0x01,1,SMB_Add)
        print(state)
    
    def status(self):
        return d.read_smb_cmd(0x01,1,SMB_Add)



    

In [15]:
check_REV()

['0x2', '0x0', '0x0', '0x4']

In [16]:
Command('avail_NVM_slots',SMB_Add).formatted()

23

In [17]:
check_ID()

True

In [18]:
load_commands(commands)

In [19]:
write_to_OTP(exec_command)

In [20]:
Command('prog_status',SMB_Add).formatted()

1

In [22]:
Command('bank_status',SMB_Add).formatted()

['0x1', '0x0', '0x0', '0x0']

In [26]:
Command('load_FW',SMB_Add).write_reg()

In [29]:
Command('recomp_OTP_bounds',SMB_Add).write_reg()


In [30]:
Command('wake_FW',SMB_Add).write_reg()

In [31]:
id = Command('config_id',SMB_Add).formatted()[0]
print(id)

0x0


In [32]:
Command('restcfg',SMB_Add).write_reg()

In [33]:
Command('avail_NVM_slots',SMB_Add).formatted()

22

In [34]:
Command('crc',SMB_Add).formatted()

['0x39', '0xc9', '0x4d', '0x13']

In [35]:
enable_pin=Enable()

In [36]:
enable_pin.on()

['0x80']
